In [3]:
import requests
import json
import pandas as pd

 - query API to get list of relevant gameIDs in date range
 - `https://www.backend.audlstats.com/api/v1/games`
   - add date range parameter, `?date=<...>`

> An inclusive date range. The full format is YYYY-MM-DD:YYYY-MM-DD, but you can exclude months or days and it will infer them. Examples:
- > 2021-06:2021-07 => All games during June or July
- > 2021 => All games during 2021
- > 2021-07-10: => All games on or after July 10th, 2021
- > :2012-05 => All games during or before May, 2012

In [25]:
URL = 'https://www.backend.audlstats.com/api/v1/games?'
parameters = 'date=2023-05-19:2023-05-19' # April and May 2023 games

In [26]:
URL = f'{URL}{parameters}'
games = requests.get(URL)
if games.status_code == 200:
    games = games.json()

In [27]:
games['data']

[{'gameID': '2023-05-19-LA-SLC',
  'awayTeamID': 'aviators',
  'homeTeamID': 'shred',
  'awayScore': 13,
  'homeScore': 25,
  'status': 'Final',
  'startTimestamp': '2023-05-19T19:00:00-06:00',
  'startTimezone': 'MDT',
  'streamingURL': 'https://audltv.vhx.tv/videos/los-angeles-at-salt-lake-5-19-2023',
  'updateTimestamp': '2023-05-29T05:43:47.960Z',
  'week': 'week-4'},
 {'gameID': '2023-05-19-OAK-POR',
  'awayTeamID': 'spiders',
  'homeTeamID': 'nitro',
  'awayScore': 24,
  'homeScore': 23,
  'status': 'Final',
  'startTimestamp': '2023-05-19T19:30:00-07:00',
  'startTimezone': 'PDT',
  'streamingURL': 'https://audltv.vhx.tv/videos/oakland-at-portland-5-19-2023',
  'updateTimestamp': '2023-05-23T18:42:42.617Z',
  'week': 'week-4'},
 {'gameID': '2023-05-19-SD-HTX',
  'awayTeamID': 'growlers',
  'homeTeamID': 'havoc',
  'awayScore': 23,
  'homeScore': 17,
  'status': 'Final',
  'startTimestamp': '2023-05-19T20:00:00-05:00',
  'startTimezone': 'CDT',
  'streamingURL': 'https://audltv.v

**Pick game and develop method to unpack events**

 - do stats derived from events match game reported stats?

In [4]:
URL = 'https://www.backend.audlstats.com/api/v1/gameEvents?'
parameters = 'gameID=2023-05-19-OAK-POR'

In [5]:
URL = f'{URL}{parameters}'
r = requests.get(URL)
if r.status_code == 200:
    data = r.json()

In [16]:
r.json()

{'object': 'object',
 'data': {'homeEvents': [{'type': 2,
    'line': ['jjohnson1',
     'efriedman',
     'jsnyder',
     'rhayes',
     'sfraner',
     'isweeney',
     'tdoi'],
    'time': 0},
   {'type': 18,
    'thrower': 'efriedman',
    'throwerX': -16.1,
    'throwerY': 7.22,
    'receiver': 'jjohnson1',
    'receiverX': -1.46,
    'receiverY': 15.54},
   {'type': 18,
    'thrower': 'jjohnson1',
    'throwerX': -1.46,
    'throwerY': 15.54,
    'receiver': 'efriedman',
    'receiverX': -17.13,
    'receiverY': 21.22},
   {'type': 18,
    'thrower': 'efriedman',
    'throwerX': -17.13,
    'throwerY': 21.22,
    'receiver': 'jsnyder',
    'receiverX': -17.45,
    'receiverY': 94.7},
   {'type': 18,
    'thrower': 'jsnyder',
    'throwerX': -17.45,
    'throwerY': 94.7,
    'receiver': 'rhayes',
    'receiverX': -17.39,
    'receiverY': 86.83},
   {'type': 18,
    'thrower': 'rhayes',
    'throwerX': -17.39,
    'throwerY': 86.83,
    'receiver': 'sfraner',
    'receiverX': -17.0

In [6]:
len(data['data']['homeEvents'])

406

In [7]:
len(data['data']['awayEvents'])

418

In [8]:
data.keys()

dict_keys(['object', 'data'])

In [35]:
data['data']['homeEvents'][0:50]

[{'type': 2,
  'line': ['jjohnson1',
   'efriedman',
   'jsnyder',
   'rhayes',
   'sfraner',
   'isweeney',
   'tdoi'],
  'time': 0,
  'desc': 'O_line'},
 {'type': 18,
  'thrower': 'efriedman',
  'throwerX': -16.1,
  'throwerY': 7.22,
  'receiver': 'jjohnson1',
  'receiverX': -1.46,
  'receiverY': 15.54,
  'desc': 'Pass'},
 {'type': 18,
  'thrower': 'jjohnson1',
  'throwerX': -1.46,
  'throwerY': 15.54,
  'receiver': 'efriedman',
  'receiverX': -17.13,
  'receiverY': 21.22,
  'desc': 'Pass'},
 {'type': 18,
  'thrower': 'efriedman',
  'throwerX': -17.13,
  'throwerY': 21.22,
  'receiver': 'jsnyder',
  'receiverX': -17.45,
  'receiverY': 94.7,
  'desc': 'Pass'},
 {'type': 18,
  'thrower': 'jsnyder',
  'throwerX': -17.45,
  'throwerY': 94.7,
  'receiver': 'rhayes',
  'receiverX': -17.39,
  'receiverY': 86.83,
  'desc': 'Pass'},
 {'type': 18,
  'thrower': 'rhayes',
  'throwerX': -17.39,
  'throwerY': 86.83,
  'receiver': 'sfraner',
  'receiverX': -17.06,
  'receiverY': 74.83,
  'desc': 'P

In [1]:
# https://www.docs.audlstats.com/events

events = [
    'D_line',
    'O_line',
    'Timeout_own_line',
    'Timeout_own',
    'Timeout_opp_line',
    'Timeout_opp',
    'Pull_in',
    'Pull_out',
    'Offsides_own',
    'Offsides_opp',
    'Block',
    'Callahan_opp',
    'Throwaway_opp',
    'Stall_opp',
    'Score_opp',
    'Penalty_own',
    'Penalty_opp',
    'Pass',
    'Goal',
    'Drop',
    'Drop_pull',
    'Throwaway_own',
    'Callahan_own',
    'Stall_own',
    'Injury_line',
    'Player_misconduct',
    'Player_ejection',
    'End_Q1',
    'End_Q2',
    'End_Q3',
    'End_Q4',
    'End_OT1',
    'End_OT2',
    'Delayed',
    'Postponed',
]

In [2]:
event_keys = {}
for i,val in enumerate(events):
    event_keys[i+1]=val

In [34]:
for etype in ['homeEvents','awayEvents']:
    for event in data['data'][etype]:
        event['desc'] = event_keys[event['type']]

In [ ]:
event_keys

NameError: name 'event_keys' is not defined

**Go through home or away events, see if I can split by each score/timeout/injury**

 - should probably characterize by O/D eventually
 - would have to cross reference home/away events to get complete information about turnovers

Possible Data's

 - line info
 - time, starts at 0 every quarter up to 720s (12 mins)
 - playerID (line arrays, puller, thrower, receiver, defender, misconduct foul, ejected)
 - pull info (`X`,`Y` for where it was brought into play, hangtime `Ms`)
 - player coordinates (thrower, receiver, turnover)


In [30]:
# line substitutions within a point, injury or timeout
for i,event in enumerate(data['data']['homeEvents']):
    if event['type'] in [3,5,25]:
        print(i,event_keys[event['type']],event)
for i,event in enumerate(data['data']['awayEvents']):
    if event['type'] in [3,5,25]:
        print(i,event_keys[event['type']],event)

67 Timeout_own_line {'type': 3, 'line': ['bstout', 'rhayes', 'qbuermeye', 'apetersch', 'jlee', 'isweeney', 'jmoore1'], 'time': 403}
177 Timeout_opp_line {'type': 5, 'line': ['sradlauer', 'jmoore1', 'jlee', 'gnakano', 'bstout', 'apetersch', 'qbuermeye'], 'time': 610}
228 Injury_line {'type': 25, 'line': ['sradlauer', 'jmoore1', 'jlee', 'gnakano', 'apetersch', 'arueda', 'bstout'], 'time': 291}
284 Timeout_own_line {'type': 3, 'line': ['jjohnson1', 'efriedman', 'jsnyder', 'rhayes', 'sfraner', 'isweeney', 'qbuermeye'], 'time': 698}
304 Injury_line {'type': 25, 'line': ['jjohnson1', 'efriedman', 'jsnyder', 'rhayes', 'isweeney', 'qbuermeye', 'tdoi'], 'time': 121}
404 Timeout_opp_line {'type': 5, 'line': ['sradlauer', 'rhayes', 'cstillwel', 'isweeney', 'bstout', 'jsnyder', 'slatarski'], 'time': 718}
36 Timeout_opp_line {'type': 5, 'line': ['rvickersb', 'pxu', 'jdecampos', 'mburke1', 'cbauman', 'rmendoza', 'wberreman'], 'time': 403}
157 Timeout_opp_line {'type': 5, 'line': ['gmay', 'rcastro', 

### Flow of a Point

*can always use point start to split points*

 1. Start point with `O_line` or `D_line`, note players on the field.
   a. adjustments needed if event type in 3,5,25. All players can be subbed on timeouts.
 2. Series of events from 7-27 (excepting Callahans), starting with 7 or 8 `Pull_in` or `Pull_out`
 3. Point ends with quarter (28-33) or some type of score (

In [42]:
# count points for home and away to check
pointcounter = {'homeEvents':{'O':0,'D':0},
                'awayEvents':{'O':0,'D':0}
               }

for etype in ['homeEvents','awayEvents']:
    for event in data['data'][etype]:
        if event['type'] < 3:
            if event['type'] == 2:
                pointcounter[etype]['O'] += 1 
            else:
                pointcounter[etype]['D'] += 1

In [43]:
pointcounter

{'homeEvents': {'O': 26, 'D': 24}, 'awayEvents': {'O': 24, 'D': 26}}

In [44]:
# attempt to number the groups of events by points for cross referencing

from collections import defaultdict

In [63]:
home_points = defaultdict(list)
k = 0

for event in data['data']['homeEvents']:
    if event['type'] < 3:
        k+=1
    home_points[k].append(event)         

In [64]:
away_points = defaultdict(list)
k = 0

for event in data['data']['awayEvents']:
    if event['type'] < 3:
        k+=1
    away_points[k].append(event)

In [54]:
k

50

### Flow of Point, considering everything

 1. Start with D, pull
   - pull in both lines
   - find which team is pulling
 2. Then go to O,
 3. Switch teams with any kind of turnover
   - turnovers (offensive perspective): 20,21,22,23\*,24
   - Drop, Dropped pull, throwaway, Callahan\*, Stall
   - How to handle "Blocks", 11?

In [69]:
home_points[1]

[{'type': 2,
  'line': ['jjohnson1',
   'efriedman',
   'jsnyder',
   'rhayes',
   'sfraner',
   'isweeney',
   'tdoi'],
  'time': 0,
  'desc': 'O_line'},
 {'type': 18,
  'thrower': 'efriedman',
  'throwerX': -16.1,
  'throwerY': 7.22,
  'receiver': 'jjohnson1',
  'receiverX': -1.46,
  'receiverY': 15.54,
  'desc': 'Pass'},
 {'type': 18,
  'thrower': 'jjohnson1',
  'throwerX': -1.46,
  'throwerY': 15.54,
  'receiver': 'efriedman',
  'receiverX': -17.13,
  'receiverY': 21.22,
  'desc': 'Pass'},
 {'type': 18,
  'thrower': 'efriedman',
  'throwerX': -17.13,
  'throwerY': 21.22,
  'receiver': 'jsnyder',
  'receiverX': -17.45,
  'receiverY': 94.7,
  'desc': 'Pass'},
 {'type': 18,
  'thrower': 'jsnyder',
  'throwerX': -17.45,
  'throwerY': 94.7,
  'receiver': 'rhayes',
  'receiverX': -17.39,
  'receiverY': 86.83,
  'desc': 'Pass'},
 {'type': 18,
  'thrower': 'rhayes',
  'throwerX': -17.39,
  'throwerY': 86.83,
  'receiver': 'sfraner',
  'receiverX': -17.06,
  'receiverY': 74.83,
  'desc': 'P

### Field Description Notes

 - home/away score is for end of event
 - quarter and time are for start of event
   - time is time left in the period (s)
 - event cols are booleans
 - team cols are home/away
 - `pullMs` is a float for inbounds pull, None for OB pull
   - same with X/Y
   - *are dropped pull coordinates (X,Y for receiver) needed in addition to pull X/Y?*


In [9]:
data

{'object': 'object',
 'data': {'homeEvents': [{'type': 2,
    'line': ['jjohnson1',
     'efriedman',
     'jsnyder',
     'rhayes',
     'sfraner',
     'isweeney',
     'tdoi'],
    'time': 0},
   {'type': 18,
    'thrower': 'efriedman',
    'throwerX': -16.1,
    'throwerY': 7.22,
    'receiver': 'jjohnson1',
    'receiverX': -1.46,
    'receiverY': 15.54},
   {'type': 18,
    'thrower': 'jjohnson1',
    'throwerX': -1.46,
    'throwerY': 15.54,
    'receiver': 'efriedman',
    'receiverX': -17.13,
    'receiverY': 21.22},
   {'type': 18,
    'thrower': 'efriedman',
    'throwerX': -17.13,
    'throwerY': 21.22,
    'receiver': 'jsnyder',
    'receiverX': -17.45,
    'receiverY': 94.7},
   {'type': 18,
    'thrower': 'jsnyder',
    'throwerX': -17.45,
    'throwerY': 94.7,
    'receiver': 'rhayes',
    'receiverX': -17.39,
    'receiverY': 86.83},
   {'type': 18,
    'thrower': 'rhayes',
    'throwerX': -17.39,
    'throwerY': 86.83,
    'receiver': 'sfraner',
    'receiverX': -17.0

In [8]:
data = data['data']

In [55]:
# initialize stuff
game_length = len(data['homeEvents']) + len(data['awayEvents'])
game_events = [val for val in range(game_length)]

table = pd.DataFrame(index = game_events,
        columns = 
    ['home_team','away_team', # other info that won't change during a game
    'home_score','away_score','quarter','time', 'point',# score / time stuff
    'pull','drop_pull','offsides','pass','drop','throwaway','stall','block','callahan', #event booleans
    'penalty','goal','injury','Q1_end','Q2_end','Q3_end','Q4_end','OT1_end','OT2_end', #events cont'd
    'game_delay','game_postponed', 'player_misconduct','player_ejected','substitutions','timeout', #events cont'd
    'team_off','team_def','team_score', 'team_penalty', 'team_timeout',  # team ID for events, infer most from O/D
    'thrower','receiver','defender','pullMs','deltaX','deltaY',]) # player ID for events, disc movement for events

table.loc[game_events,'home_team'] = 'POR'
table.loc[game_events,'away_team'] = 'OAK'
table.loc[0,'home_score'] = 0
table.loc[0,'away_score'] = 0
table.loc[0,'quarter'] = 1
table.loc[0,'point'] = 1
table.loc[0,'time'] = 720
#table.loc[0,'pull'] = True

In [57]:
homes = data['homeEvents']
aways = data['awayEvents']

for home in homes[0:10]:
    print(event_keys[home['type']], home)
print()
for home in aways[0:3]:
    print(event_keys[home['type']], home)

O_line {'type': 2, 'line': ['jjohnson1', 'efriedman', 'jsnyder', 'rhayes', 'sfraner', 'isweeney', 'tdoi'], 'time': 0}
Pass {'type': 18, 'thrower': 'efriedman', 'throwerX': -16.1, 'throwerY': 7.22, 'receiver': 'jjohnson1', 'receiverX': -1.46, 'receiverY': 15.54}
Pass {'type': 18, 'thrower': 'jjohnson1', 'throwerX': -1.46, 'throwerY': 15.54, 'receiver': 'efriedman', 'receiverX': -17.13, 'receiverY': 21.22}
Pass {'type': 18, 'thrower': 'efriedman', 'throwerX': -17.13, 'throwerY': 21.22, 'receiver': 'jsnyder', 'receiverX': -17.45, 'receiverY': 94.7}
Pass {'type': 18, 'thrower': 'jsnyder', 'throwerX': -17.45, 'throwerY': 94.7, 'receiver': 'rhayes', 'receiverX': -17.39, 'receiverY': 86.83}
Pass {'type': 18, 'thrower': 'rhayes', 'throwerX': -17.39, 'throwerY': 86.83, 'receiver': 'sfraner', 'receiverX': -17.06, 'receiverY': 74.83}
Pass {'type': 18, 'thrower': 'sfraner', 'throwerX': -17.06, 'throwerY': 74.83, 'receiver': 'efriedman', 'receiverX': 0.35, 'receiverY': 63.8}
Pass {'type': 18, 'thro

In [ ]:
    ['home_team','away_team', # other info that won't change during a game, initialize before looping through events
    'home_score','away_score','quarter','time', 'point',# score / time stuff
    'pull','drop_pull','offsides','pass','drop','throwaway','stall','block','callahan', #event booleans
    'penalty','goal','injury','Q1_end','Q2_end','Q3_end','Q4_end','OT1_end','OT2_end', #events cont'd
    'game_delay','game_postponed', 'player_misconduct','player_ejected','substitutions','timeout', #events cont'd
    'team_off','team_def','team_score', 'team_penalty', 'team_timeout',  # team ID for events, infer most from O/D
    'thrower','receiver','defender','X','Y', # player ID for events, disc movement for events
    'pullMs','deltaX','deltaY']) # player ID for events, disc movement for events

In [56]:
table.head()

,home_team,away_team,home_score,away_score,quarter,time,point,pull,drop_pull,offsides,...,team_def,team_score,team_penalty,team_timeout,thrower,receiver,defender,pullMs,deltaX,deltaY
0,POR,OAK,0,0,1,720,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POR,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POR,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POR,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POR,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
home_team = 'POR'
away_team = 'OAK'
team_list = [home_team, away_team]

In [ ]:
# bring events into lists, to be popped
event_lists = {home_team: data['homeEvents'], away_team: data['awayEvents']}
pos_team = None
puller = None
def_team = None

# pull
for i in game_events[0:]:
    
    home_type = event_lists[home_team][0]['type']
    away_type = event_lists[away_team][0]['type']
    
    # period start: O/D line classification
    if not pos_team:
        if home_type == 2:
            table.loc[i,'team_off'] = home_team
            table.loc[i,'team_def'] = away_team
            puller = away_team
            pos_team = home_team
        else:
            table.loc[i,'team_off'] = away_team
            table.loc[i,'team_def'] = home_team
            puller = home_team
            pos_team = away_team
        for val in team_list:
            event_lists[val].pop(0)
    
    # pull
    if puller:
        # pull stats
        table.loc[i,'pull'] = True
        pull = event_lists[puller].pop(0)
        table.loc[i,'thrower'] = pull.get('puller')
        table.loc[i,'pullMs'] = pull.get('pullMs')
        table.loc[i,'deltaX'] = pull.get('pullX')
        table.loc[i,'deltaY'] = pull.get('pullY')
        puller = None
        
        # offsides check should happen
        
        # dropped pull?
        if event_lists[pos_team][0]['type'] == 21:
            print('dropped pull should do stuff!!!')
            break
            
        def_team = away_team
        continue
        
    # turnover keys, score keys, callahan keys
    # 11, 13, 14, 20, 22, 24
    # 15, 19
    # 12, 23, 
    
    # period stoppage, timeout, other
    # 28,29,30,31,32,33
    # 3,4,5,6
    # 34,35
    
    
    
        

In [58]:
event_keys

{1: 'D_line',
 2: 'O_line',
 3: 'Timeout_own_line',
 4: 'Timeout_own',
 5: 'Timeout_opp_line',
 6: 'Timeout_opp',
 7: 'Pull_in',
 8: 'Pull_out',
 9: 'Offsides_own',
 10: 'Offsides_opp',
 11: 'Block',
 12: 'Callahan_opp',
 13: 'Throwaway_opp',
 14: 'Stall_opp',
 15: 'Score_opp',
 16: 'Penalty_own',
 17: 'Penalty_opp',
 18: 'Pass',
 19: 'Goal',
 20: 'Drop',
 21: 'Drop_pull',
 22: 'Throwaway_own',
 23: 'Callahan_own',
 24: 'Stall_own',
 25: 'Injury_line',
 26: 'Player_misconduct',
 27: 'Player_ejection',
 28: 'End_Q1',
 29: 'End_Q2',
 30: 'End_Q3',
 31: 'End_Q4',
 32: 'End_OT1',
 33: 'End_OT2',
 34: 'Delayed',
 35: 'Postponed'}

In [63]:
x = [0,1]

## Event Classifer Class
#### initialize with: gameinfo (home,away), event lists, eventually big-scheme index
#### Based on event key . . .
#### add certain stuff to table from home/away events
#### setup things for next event
#### continue through loop

In [31]:
games['data'][1:2]

[{'gameID': '2023-05-19-OAK-POR',
  'awayTeamID': 'spiders',
  'homeTeamID': 'nitro',
  'awayScore': 24,
  'homeScore': 23,
  'status': 'Final',
  'startTimestamp': '2023-05-19T19:30:00-07:00',
  'startTimezone': 'PDT',
  'streamingURL': 'https://audltv.vhx.tv/videos/oakland-at-portland-5-19-2023',
  'updateTimestamp': '2023-05-23T18:42:42.617Z',
  'week': 'week-4'}]

In [85]:
sum([len(testgame.gamedata[val]) for val in testgame.gamedata.keys()])

824

In [100]:
testgame.gamedata[testgame.away][1]

{'type': 7,
 'puller': 'cbauman',
 'pullX': -19.58,
 'pullY': 110.93,
 'pullMs': 3879}

In [98]:
testgame.gamedata[testgame.home][0:5]

[{'type': 2,
  'line': ['jjohnson1',
   'efriedman',
   'jsnyder',
   'rhayes',
   'sfraner',
   'isweeney',
   'tdoi'],
  'time': 0},
 {'type': 18,
  'thrower': 'efriedman',
  'throwerX': -16.1,
  'throwerY': 7.22,
  'receiver': 'jjohnson1',
  'receiverX': -1.46,
  'receiverY': 15.54},
 {'type': 18,
  'thrower': 'jjohnson1',
  'throwerX': -1.46,
  'throwerY': 15.54,
  'receiver': 'efriedman',
  'receiverX': -17.13,
  'receiverY': 21.22},
 {'type': 18,
  'thrower': 'efriedman',
  'throwerX': -17.13,
  'throwerY': 21.22,
  'receiver': 'jsnyder',
  'receiverX': -17.45,
  'receiverY': 94.7},
 {'type': 18,
  'thrower': 'jsnyder',
  'throwerX': -17.45,
  'throwerY': 94.7,
  'receiver': 'rhayes',
  'receiverX': -17.39,
  'receiverY': 86.83}]

#### Rules Notes, [link](https://theaudl.com/sites/default/files/AUDLRuleBook2023.pdf)
 - flip determines starting O/D and sides of field
   - 4 quarters, 12 mins, 2 timeouts per half with subs allowed
 - decisions reverse at the start of each quarter
 - additional flip for overtime, decsisions revers if 2nd overtime needed
   - OT1 is 5 mins, OT2 is sudden death without time
   - 1 timeout per overtime
 - see **17 Glossary** for definitions of events

In [92]:
pd.DataFrame(testgame.table.iloc[0,4:]).T

,away_team,home_score,away_score,quarter,time,point,pull,drop_pull,offsides,pass,...,team_score,team_penalty,team_timeout,penalty_player,thrower,receiver,defender,pullMs,deltaX,deltaY
0,OAK,0,0,1,720,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
class Parser:
    def __init__(self, gameinfo, gamedata): # provide game metadata, and gamedata['data']
        
        self.gameinfo = gameinfo
        game_length = len(gamedata['homeEvents']) + len(gamedata['awayEvents']) # prepopulate index with max events
        
        # use team abbreviations
        away_team = gameinfo['gameID'].split('-')[-2]
        self.away = away_team
        home_team = gameinfo['gameID'].split('-')[-1]
        self.home = home_team

        # gather home and away events into lists, to be popped
        self.gamedata = {home_team: gamedata['homeEvents'], away_team: gamedata['awayEvents']}

        # intialize table
        table = pd.DataFrame(index = [val for val in range(game_length)],
                columns = 
            ['gameID','date','week','home_team','away_team', # flat attributes for the game
            'home_score','away_score','quarter','time', 'point', # score / time
            'pull','drop_pull','offsides','pass','drop','throwaway','stall','block','callahan', #event booleans
            'penalty','goal','injury','Q1_end','Q2_end','Q3_end','Q4_end','OT1_end','OT2_end', #events cont'd
            'game_delay','game_postponed', 'player_misconduct','player_ejected','substitutions','timeout', #events cont'd
            'team_off','team_def','team_score', 'team_penalty', 'team_timeout',  # team ID for events, infer most from O/D
            'penalty_player','thrower','receiver','defender', # player ID for events
            'pullMs','deltaX','deltaY','X','Y',]) # disc movement, player position for events

        table.loc[:,'gameID'] = gameinfo['gameID']
        table.loc[:,'date'] = f"{gameinfo['startTimestamp']}_{gameinfo['startTimezone']}"
        table.loc[:,'week'] = gameinfo['week'].replace('week-','').replace('-weekend','')     
        table.loc[:,'home_team'] = home_team
        table.loc[:,'away_team'] = away_team
        
        table.loc[0,'home_score'] = 0
        table.loc[0,'away_score'] = 0
        table.loc[0,'quarter'] = 1
        table.loc[0,'point'] = 1
        table.loc[0,'time'] = 720
        
        self.table = table
        self.ind = 0
        
    
    def pos_change(pos_team):
        return self.home if pos_team != self.home else self.away
    
    def parse_events(self):

        # start of game, determine starting team
        event_home = self.gamedata[self.home].pop(0)
        home_line = event_home['line']
        event_away = self.gamedata[self.away].pop(0)
        away_line = event_away['line']
        
        # write first record, collect "startO" for quarter changes
        if event_home['type'] == 2: # 2 is starting O line, 1 is starting D line
            self.table.loc[0,'team_off'] = home_team
            self.table.loc[0,'team_def'] = away_team
            pos_team = self.home
            startO = self.home
        else:
            self.table.loc[0,'team_off'] = away_team
            self.table.loc[0,'team_def'] = home_team
            pos_team = self.away
            startO = self.away


        # go through all events in list
        while sum([len(self.gamedata[val]) for val in self.gamedata.keys()]) > 0:

            if self.ind != 0:
                # get lines again           
                home_line = ...
                away_line = ...
            
            # pull
            event = self.gamedata[pos_change(pos_team)].pop(0) # pop
            
            if event['type'] not in [7,8]: # check
                print('NOT A PULLER !!!', event)
                break 
            
            self.table.loc[self.ind,'pull'] = 1
            self.table.loc[self.ind,'thrower'] = event['puller']
            if event['type'] == 7:
                self.table.loc[self.ind,'pullMs'] = event['pullMs']
                self.table.loc[self.ind,'deltaX'] = event['pullX']
                self.table.loc[self.ind,'deltaY'] = event['pullY']
            
            self.ind += 1 # new event
            
            # pull reception
            event = self.gamedata[pos_team][0]
            
            # OFFSIDES CHECK ???
            
            if event['type'] == 21: # pull was dropped    
                event = self.gamedata[pos_team.pop(0)] # pop
                self.table.loc[self.ind, 'drop_pull'] = 1
                self.table.loc[self.ind, 'receiver'] = event['receiver']
                self.table.loc[self.ind, 'X'] = event['receiverX']
                self.table.loc[self.ind, 'Y'] = event['receiverY']
                pos_team = pos_change(pos_team)
                self.ind += 1 # new event
            
            event = self.gamedata[pos_team][0]
            while event['type'] != 19 or event['type'] > 27: # score or end of period
                
   
            
            # events until turnover
            # ...
    # turnover keys, score keys, callahan keys
    # 11, 13, 14, 20, 22, 24
    # 15, 19
    # 12, 23, 
    
    # period stoppage, timeout, other
    # 28,29,30,31,32,33
    # 3,4,5,6
    # 34,35
            # goal
            # restart loop

    
    def write_record(self):
        return


In [ ]:
        
    # def event_classifier(event_type):
    #     match event_type:
    #         case in [1,2]:
    #             # OLine/Dline
    #                 # logic for Line Starts
    #                 # include pull 7-8, offsides 9-10
    #                 # dropped pull covered in turnovers 21
    #         case 18: 
    #             # Completed Pass
    #                 # logic for a completed pass
    #         case in [15,19]:
    #             # Score_opp, Goal
    #                 # logic for score
    #         case in [11,13,14,20,21,22,24]:
    #             # block, throwaway, stall, drop
    #                 # turnover logic
    #         case in [16,17]:
    #             # penalty
    #         case >= 28:
    #             # quarter ends and special delays
    #         case in [3,4,5,6,25]:
    #             # timeout or injury
    #         case in [12,23]:
    #             # callahan
    #         case in [26,27]:
    #             # special foul

In [72]:
URL = 'https://www.backend.audlstats.com/api/v1/gameEvents?'
gamedata = None

for game in games['data'][1:2]:
    # request game events
    parameters = 'gameID=2023-05-19-OAK-POR'

    URL = f'{URL}gameID={game["gameID"]}'
    r = requests.get(URL)
    if r.status_code == 200:
        gamedata = r.json()
        gamedata = gamedata.get('data')
    if not gamedata:
        print('error for', game['gameID'])
        break
        
    testgame = Parser(game, gamedata)

In [84]:
testgame.gamedata

{'POR': [{'type': 2,
   'line': ['jjohnson1',
    'efriedman',
    'jsnyder',
    'rhayes',
    'sfraner',
    'isweeney',
    'tdoi'],
   'time': 0},
  {'type': 18,
   'thrower': 'efriedman',
   'throwerX': -16.1,
   'throwerY': 7.22,
   'receiver': 'jjohnson1',
   'receiverX': -1.46,
   'receiverY': 15.54},
  {'type': 18,
   'thrower': 'jjohnson1',
   'throwerX': -1.46,
   'throwerY': 15.54,
   'receiver': 'efriedman',
   'receiverX': -17.13,
   'receiverY': 21.22},
  {'type': 18,
   'thrower': 'efriedman',
   'throwerX': -17.13,
   'throwerY': 21.22,
   'receiver': 'jsnyder',
   'receiverX': -17.45,
   'receiverY': 94.7},
  {'type': 18,
   'thrower': 'jsnyder',
   'throwerX': -17.45,
   'throwerY': 94.7,
   'receiver': 'rhayes',
   'receiverX': -17.39,
   'receiverY': 86.83},
  {'type': 18,
   'thrower': 'rhayes',
   'throwerX': -17.39,
   'throwerY': 86.83,
   'receiver': 'sfraner',
   'receiverX': -17.06,
   'receiverY': 74.83},
  {'type': 18,
   'thrower': 'sfraner',
   'throwerX

In [83]:
testgame.table.head()

,gameID,date,week,home_team,away_team,home_score,away_score,quarter,time,point,...,team_score,team_penalty,team_timeout,penalty_player,thrower,receiver,defender,pullMs,deltaX,deltaY
0,2023-05-19-OAK-POR,2023-05-19T19:30:00-07:00_PDT,4,POR,OAK,0,0,1,720,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-05-19-OAK-POR,2023-05-19T19:30:00-07:00_PDT,4,POR,OAK,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-05-19-OAK-POR,2023-05-19T19:30:00-07:00_PDT,4,POR,OAK,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-05-19-OAK-POR,2023-05-19T19:30:00-07:00_PDT,4,POR,OAK,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-05-19-OAK-POR,2023-05-19T19:30:00-07:00_PDT,4,POR,OAK,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Most common events: [game stats](https://theaudl.com/stats/game/2023-05-19-OAK-POR)
 - throw
 - point starts
 - pull, score, goal
 - throwaway, block
 - rest

In [12]:
# count each event, make match classifer more efficient by trying most common first
event_counts = {}
for i,event in enumerate(data['homeEvents']):
    if f"{event['type']}, {event_keys[event['type']]}" in event_counts.keys():
        event_counts[f"{event['type']}, {event_keys[event['type']]}"] += 1
    else:
        event_counts[f"{event['type']}, {event_keys[event['type']]}"] = 1
for i,event in enumerate(data['awayEvents']):
    if f"{event['type']}, {event_keys[event['type']]}" in event_counts.keys():
        event_counts[f"{event['type']}, {event_keys[event['type']]}"] += 1
    else:
        event_counts[f"{event['type']}, {event_keys[event['type']]}"] = 1

In [15]:
sorted(event_counts.items(), key=lambda x:x[1], reverse=True)

[('18, Pass', 464),
 ('2, O_line', 50),
 ('1, D_line', 50),
 ('19, Goal', 47),
 ('15, Score_opp', 47),
 ('7, Pull_in', 46),
 ('22, Throwaway_own', 31),
 ('11, Block', 21),
 ('17, Penalty_opp', 15),
 ('13, Throwaway_opp', 14),
 ('16, Penalty_own', 10),
 ('5, Timeout_opp_line', 5),
 ('3, Timeout_own_line', 4),
 ('20, Drop', 4),
 ('25, Injury_line', 4),
 ('8, Pull_out', 4),
 ('28, End_Q1', 2),
 ('29, End_Q2', 2),
 ('30, End_Q3', 2),
 ('31, End_Q4', 2)]

In [101]:
# investigate various events
for i,event in enumerate(data['homeEvents']):
    if event['type'] in [11,14,24]:
        print(i,event_keys[event['type']],event)
for i,event in enumerate(data['awayEvents']):
    if event['type'] in [11,14,24]:
        print(i,event_keys[event['type']],event)

23 Block {'type': 11, 'defender': 'sradlauer'}
154 Block {'type': 11, 'defender': 'sradlauer'}
255 Block {'type': 11, 'defender': 'rhayes'}
266 Block {'type': 11, 'defender': 'gnakano'}
299 Block {'type': 11, 'defender': 'efriedman'}
308 Block {'type': 11, 'defender': 'sradlauer'}
343 Block {'type': 11, 'defender': 'llavoie'}
369 Block {'type': 11, 'defender': 'arueda'}
387 Block {'type': 11, 'defender': 'jmoore1'}
2 Block {'type': 11, 'defender': 'rvickersb'}
14 Block {'type': 11, 'defender': 'emagsig'}
18 Block {'type': 11, 'defender': 'dclyburn'}
21 Block {'type': 11, 'defender': 'emagsig'}
128 Block {'type': 11, 'defender': 'rmendoza'}
140 Block {'type': 11, 'defender': 'mhecht'}
145 Block {'type': 11, 'defender': 'wberreman'}
151 Block {'type': 11, 'defender': 'rvickersb'}
158 Block {'type': 11, 'defender': 'rvickersb'}
184 Block {'type': 11, 'defender': 'rmendoza'}
302 Block {'type': 11, 'defender': 'mcrawford'}
341 Block {'type': 11, 'defender': 'klaurence'}
